<a href="https://colab.research.google.com/github/yanisrem/AssignmentsSB/blob/main/DM2_ajoutsPG2511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
############
# Packages #
############
import time as t
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import plotly.express as px

from scipy.stats import rv_discrete, multivariate_normal
###################
# hyperparametres #
###################

# for reproductibility
seed = 0

# data dimensions
T=200
k=100
l = 1

#Z : (k,1)
#Y : (T,1)
#U : (T,l)
#X : (T,k)

# data generations

# for X:
rho=0.75

# for Beta : number of non null
s=5 #in [5,10,100]
lst_s = [5,10,100]

# for sigma2 :ratio between explained and total variance
Ry=0.02 #in [0.02, 0.25, 0.5]
lst_Ry = [0.02, 0.25, 0.5]

# for q prior
a=1
b=1

# for R2 prior
A=1
B=1

### Initialize parameters

Firstly, let's comput the initialization of the parameters

In [75]:
def compute_X(T, k, rho):
    """Compute matrix of xt observations

    Args:
        T (int): number of observations
        k (int): number of predictors
        rho (float): Toeplitz correlation parameter

    Returns:
        np.array: dimensions T*k
    """
    cov_matrix=np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            cov_matrix[i,j]=rho**np.abs(i-j)
    return np.random.multivariate_normal([0]*k, cov_matrix, T)

def compute_U(T, l, rho):
    """Compute matrix of xt observations

    Args:
        T (int): number of observations
        l (int): number of predictors
        rho (float): Toeplitz correlation parameter

    Returns:
        np.array: dimensions T*l
    """
    cov_matrix=np.zeros((l, l))
    for i in range(l):
        for j in range(l):
            cov_matrix[i,j]=rho**np.abs(i-j)
    return np.random.multivariate_normal([0]*l, cov_matrix, T)

def sample_phi(l):
    return np.random.uniform(0,1, size=l)

def compute_vx(X):
    return np.mean(np.var(X,axis=0))

def sample_beta(k, s):
    """Sample of beta vector of dimensions 1*k

    Args:
        k (int): number of predictors
        s (int): number of non-zero elements of beta

    Returns:
        np.array: dimensions 1*k
    """
    beta=np.zeros(k)
    index_normal_distribution=np.random.choice(len(beta), size=s, replace=False)
    beta[index_normal_distribution] = np.random.normal(loc=0, scale=1, size=s)
    return beta

def comput_Z(beta):
    Z=beta
    Z[Z!=0]=1
    return Z

def compute_sigma2(Ry, beta, X):
    """ Compute sigma2
    Args:
        Ry (float): pourcentage of explained variance
        beta (np.array): beta previously sampled
        X (np.array): matrix of (xt) samples

    Returns:
        float: dimensions 1*1
    """
    return (1/Ry-1)*np.mean(np.square(X @ beta))

def compute_R2(q, k, gamma2, v_x):
    return (q*k*gamma2*v_x)/(q*k*gamma2*v_x+1)

def sample_epsilon(T, sigma2):
    """Sample epsilon_1,...,epsilon_T

    Args:
        T (int): number of observations
        sigma2 (float): sigma2 previously sampled

    Returns:
        np.array: dimensions 1*T
    """
    return np.random.normal(loc=0, scale=sigma2, size=T)

def compute_Y(X, beta, epsilon):
    return X@beta + epsilon


### Final function
def init_parameters(seed, T, k, l, rho, s, Ry, a, b, A, B):
    """
    Initialize parameters for a given simulation.

    Args:
        seed (int): Seed for reproducibility.
        T (int): Number of observations.
        k (int): Number of covariates.
        l (int): Number of latent variables.
        rho (float): Correlation parameter.
        s (float): Scaling parameter.
        Ry (float): Response variance.
        a (float): Shape parameter for gamma2.
        b (float): Shape parameter for gamma2.
        A (float): Shape parameter for q.
        B (float): Shape parameter for q.

    Returns:
        dict: Dictionary containing initialized parameters.
    """
    np.random.seed(seed=seed)
    dct = {
        "X" : compute_X(T=T, k=k, rho=rho),
        "U": compute_U(T=T, l=l, rho=rho),
        "Beta": sample_beta(k=k, s=s),
        "phi": sample_phi(l=l),
        "q": np.random.beta(A,B),
        "gamma2": np.random.beta(a,b),
    }
    dct["vx"] = compute_vx(X=dct["X"])
    dct["R2"] = compute_R2(q=dct["q"], k=k, gamma2=dct["gamma2"], v_x=dct["vx"])
    dct["Z"]=comput_Z(beta=dct["Beta"])
    dct["sigma2"] = compute_sigma2(Ry=Ry, beta=dct["Beta"], X=dct["X"])
    dct["epsilon"] = sample_epsilon(T=T, sigma2=dct["sigma2"])
    dct["Y"]=compute_Y(X=dct["X"], beta=dct["Beta"], epsilon=dct["epsilon"])
    return dct


[-1.80786429e+02 -4.94156044e+01  2.54491549e-01  3.12087518e+02
 -1.59445254e+02  3.32337440e+02  4.87565338e+01  9.28107568e+01
  4.89453650e+01  1.33436116e+02  1.54978691e+02  5.90113668e+00
 -1.66999144e+02  6.17807155e+01  6.69381162e+02 -1.08885567e+02
  8.74436846e+01 -2.83014906e+02 -9.35845049e+01 -3.92315362e+01
  2.40932140e+02 -1.30093495e+02  1.90485234e+01 -1.46083747e+02
  1.27621624e+02  1.10897479e+02 -1.50011686e+02 -4.54141583e+02
  2.90409169e+02  2.92929829e+02 -3.12304866e+02  8.20928916e+01
  1.02085690e+02 -2.87849448e+02 -3.33848341e+02  7.85407953e+01
  4.58387790e+02  3.23448601e+02 -1.78534082e+02  5.25184039e+02
 -1.89372700e+02  7.21886724e+01 -2.55178930e+02  7.21271008e+00
 -3.01972943e+02  1.21784621e+02  1.72390477e+02 -1.02568263e+02
 -6.05104018e+01 -4.04664994e+02 -1.16296637e+02  1.13468165e+02
  2.85634769e+02  6.58079004e+01  3.91163134e+01 -1.18472357e+02
  1.83060522e+02  1.43173576e+02 -7.63270833e+01 -2.83301975e+02
 -1.56081006e+02 -2.03354

In [7]:
dct = init_parameters(seed,T,k,l,rho,s,Ry,a,b,A,B)

for scalar in ["q", "gamma2", "vx", "R2", "sigma2"]:
    value = dct[scalar]
    print(f"{scalar}={value}\n")

for mat in ["Beta","phi", "Y", "epsilon", "X", "U"]:
    value = dct[mat]
    fig = px.histogram(value, histnorm='probability density', title = f"<b>Histogram of {mat}</b> shape : {value.shape} ", template="plotly_dark")
    fig.show()

q=0.8786615266594074

gamma2=0.7311313614476556

vx=0.9801051948315735

R2=0.9843661140221802

sigma2=258.5383127472955



## Sample Conditional laws

#### Sampling from discrete variables

In [8]:
def sample_discrete(seed, values, weights, n_points):
    probs = weights/weights.sum()
    return rv_discrete(seed = seed, values = (values, probs)).rvs(size=n_points)

#print(sample_discrete(seed, np.arange(3), np.arange(1,4) , 100))

def sample_discrete_ndim(seed, values, weights, n_points):
    nval = np.multiply(values.shape)
    return sample_discrete(seed, values.reshape(nval), weights.reshape(nval), n_points)

test_multy_sample = sample_discrete(seed, np.arange(27).reshape((3,3,3)), np.arange(1,28).reshape((3,3,3)) , 10000)

fig = px.histogram(test_multy_sample, histnorm='probability density', title = f"<b>Histogram from a linear 3D density</b>", template="plotly_dark")
fig.show()

### 1. Sample from the conditional posterior of $R_2$ and $q$

In [9]:
def density_unormalized_R2_q_by_Y_U_X_theta_z(R2, q, dct, k, a, b, A, B):
    vx = dct["vx"]
    s_z = np.sum(dct["Z"])
    sigma2 = dct["sigma2"]
    Beta = dct["Beta"]
    exponent = - np.prod([
        1/(1e-6 + 2*sigma2),
        (k*vx*q*(1-R2))/(1e-6 + R2),
         np.dot(Beta, np.dot(np.diag(dct["Z"]), Beta))
        ])
    return np.prod([
        np.exp(exponent),
        q**(s_z+s_z/2+a-1),
        (1-q)**(k-s_z+b-1),
        R2**(A-1-s_z/2),
        (1-R2)**(s_z/2+B-1)
    ])

def sample_R2_q_by_Y_U_X_theta_z(seed, n_points, dct, k, a, b, A, B):

    arr0 = np.arange(0.001,0.101,0.001) # commence pas à 0 car division par 0 sinon
    arr1 = np.arange(0.11,0.91,0.01)
    arr2 = np.arange(0.901,1.001,0.001)
    discretization = np.concatenate((arr0, arr1, arr2), axis=0)

    values = np.dstack(np.meshgrid(discretization, discretization)).reshape(-1, 2)
    def density(R2_q):
        R2 = R2_q[0]
        q = R2_q[1]
        return density_unormalized_R2_q_by_Y_U_X_theta_z(R2, q, dct, k, a, b, A, B)

    weights = np.apply_along_axis(density, 1, values)
    index = np.arange(len(weights))
    sample_mask = sample_discrete(seed, index, weights, n_points)
    return values[sample_mask]

In [ ]:
fig = px.histogram(sample_R2_q_by_Y_U_X_theta_z(seed, 10000, dct, k, a, b, A, B),
                   histnorm='probability density',
                   title = f"<b>Fisrt sample from the conditional posterior of R2 and q</b> <br>R2 in blue and q in red",
                   template="plotly_dark",
                  )
fig.show()

## 2. Sample from the conditional posterior of $\phi$

In [36]:
def sample_phi_posterior(seed, dct, n_points):
    np.random.seed(seed)
    U=dct['U']
    return np.random.multivariate_normal(
        np.linalg.inv(U.T@U)@U.T@(dct['Y']-dct['X']@dct["Beta"]),
        dct["sigma2"]*np.linalg.inv(U.T@U),
        n_points
    )

In [37]:
fig = px.histogram(sample_phi_posterior(seed, dct, 10000),
                   histnorm='probability density',
                   title = f"<b>Fisrt sample from the conditional posterior of Phi</b> <br>R2 in blue and q in red",
                   template="plotly_dark",
                  )
fig.show()

### 3. Sample from the conditional posterior of $z$

#### Preliminary step: compute some random vectors

In [87]:
def compute_X_tilde(X,beta):
    """Compute T*s(z) \Tilde{X} matrix

    Args:
        X (np.array): T*p matrix of x predictors
        beta (np.array): 1*p vector of beta prior

    Returns:
        np.array: \Tilde{X} matrix
    """
    non_zero_beta=list(np.nonzero(beta)[0])
    X_tilde=X[:, non_zero_beta]
    return X_tilde

#X_tilde = compute_X_tilde(dct["X"],beta_ini)
#print(X_tilde)
def compute_W_tilde(X_tilde, gamma2):
    """Compute s(z)*s(z) \Tilde{W} matrix

    Args:
        X_tilde (np.array): T*s(z) \Tilde{X} matrix
        gamma2 (float): gamma^2

    Returns:
        np.array: \Tilde{W} matrix
    """
    I_s_z=np.identity(X_tilde.shape[1])
    return X_tilde.T@X_tilde+(1/gamma2)*I_s_z

#W_tilde = compute_W_tilde(X_tilde,gamma2)
#print(W_tilde)
def compute_Y_tilde(Y, U, phi):
    """Compute 1*T \Tilde{Y} matrix

    Args:
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        phi (np.array): 1*l phi vector

    Returns:
        np.array: \Tilde{Y}
    """
    return Y-U@phi
#Y_tilde = compute_Y_tilde(dct["Y"],dct["U"],dct["phi"])
def compute_estimator_beta_tilde(W_tilde, X_tilde, Y_tilde):
    """Compute 1*s(z) \hat{\Tilde{\beta}} vector

    Args:
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        X_tilde (np.array): T*s(z) \Tilde{X} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix

    Returns:
        np.array: 1*s(z) \hat{\Tilde{\beta}} vector
    """
    return np.linalg.inv(W_tilde)@X_tilde.T@Y_tilde
#estimator_beta_tilde = compute_estimator_beta_tilde(W_tilde,X_tilde,Y_tilde)
#print(estimator_beta_tilde)
#print(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)

In [23]:
##Beta initialized by lasso

from sklearn.linear_model import Lasso

model = Lasso(alpha=1.0)
mfit = model.fit(dct["X"],dct["Y"])
beta_ini = model.coef_
print(beta_ini)
beta_ini.size

[ -80.40170169   61.97071149   14.67184607   -2.16446318   19.1743428
  -66.52907578  -41.90760154   59.65213268  -20.19671498  -21.24000385
   27.02340316   -0.           18.41347417  -32.73503435    0.
   26.86794917  -20.24423771  -42.09374333   82.88530444   15.25402139
  -60.22264958   27.51575063   20.78957645  -21.64876954  -30.76368118
   -2.32523291   60.29920393    0.         -116.08787152   88.49545804
    5.6069285   -82.26719119  114.50469497  -28.48145399  -31.21397265
    8.35383985   44.58289355  -42.09538078  -11.16497872    0.
    3.76446734    4.45865747   26.26026857  -26.2485947    22.98544879
  -15.19439605   31.66957104   21.86594412   36.91609736    0.
  -77.37558952   14.44392383   65.797829    -33.51606187  -38.80942847
   52.70815466  -28.33759249   13.66948422    6.77399823   20.80355942
   -0.92807006   -6.06713026  -25.23536864   35.89045941  -22.81020699
   34.08896492  -53.14809835  -31.82627445   36.48942381    0.
  -97.08194318   42.12998481    7.73785

100

In [39]:
#Initialization of z
z_ini = np.zeros(100)
for i in range(0,100):
  if (beta_ini[i] != 0) :
    z_ini[i] = 1



#### Gibbs sampler

In [158]:
def compute_z_posterior(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T):
    """Compute \pi(z|Y, U, X, \phi, R^2, q) mass function

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples

    Returns:
        float: value of \pi(z|Y, U, X, \phi, R^2, q)
    """
    s_z=np.sum(Z)

    return q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-T/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)**(-T/2)*sp.special.gamma(T/2)


def compute_zi_posterior_conditional_z_excluded_i(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, zi, index):
    """Compute \pi(z_i|Y, X, U, \phi, R^2, q z_{-i}) for a specific zi

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        zi (int): Z_i variable
        index (int): index of Z_i variable in Z array

    Returns:
        float: value of \pi(z_i|Y, X, U, \phi, R^2, q z_{-i})
    """
    Z_i=np.copy(Z)
    Z_i[index]=zi
    z_posterior=compute_z_posterior(q=q, k=k, Z=Z_i, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)

    Z_0=np.copy(Z)
    Z_0[index]=0
    z_posterior_zi_equal_0=compute_z_posterior(q=q, k=k, Z=Z_0, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    Z_1=np.copy(Z)
    Z_1[index]=1
    z_posterior_zi_equal_1=compute_z_posterior(q=q, k=k, Z=Z_1, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    return -+z_posterior/(z_posterior_zi_equal_0+z_posterior_zi_equal_1)

def simulated_zi_posterior_conditional_z_excluded_i(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, index, seed):
    """Sample z_i|Y, X, U, \phi, R^2, q z_{-i} by inverse CDF method

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        index (int): index of Z_i variable in Z array
        seed (int): seed

    Returns:
        int: value of z_i|Y, X, U, \phi, R^2, q z_{-i} in {0,1}
    """
    np.random.seed(seed)
    proba_success=compute_zi_posterior_conditional_z_excluded_i(q=q, k=k, Z=Z, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, zi=1, index=index)
    u=np.random.uniform(0,1);
    if u<1-proba_success:
        return 0
    else:
        return 1

def step_gibbs_sampler_z_posterior(q, k, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, seed, Z_t_minus_1):
    """One iteration of Gibbs sampler

    Args:
        q (float): q variable
        k (int): number of x predictors
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        seed (int): seed
        Z_t_minus_1 (_type_): Z_{t-1} variable sampled during the previous iteration (t-1 step)

    Returns:
        int:
    """
    Z_t=np.copy(Z_t_minus_1)
    for i in range(len(Z_t)):
        Z_t[i]=simulated_zi_posterior_conditional_z_excluded_i(q=q, k=k, Z=Z_t, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, index=i, seed=seed)

    return Z_t

def gibbs_sampler_z_posterior(q, k, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, seed, n_iter, n_variables):
    """Gibbs sampler to simulate 1*k vectors z|Y, U, X, \phi, R^2

    Args:
        q (float): q variable
        k (int): number of x predictors
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        seed (int): seed
        n_iter (int): number of iterations
        n_variables (int): number of variables desired

    Returns:
        np.array: array of n_variables z=(z_1,...,z_k)
    """
    array_Z=[]
    for n in range(n_variables):
        Z_0=np.random.binomial(n=1, p=q, size=k) #Yanis; je ne sais pas trop comment initialiser le premier Z, j'ai repris la loi a priori
        Z_t=Z_0
        for t in range(n_iter):
            Z_t=step_gibbs_sampler_z_posterior(q=q, k=k, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, seed=seed, Z_t_minus_1=Z_t)
        array_Z.append(Z_t)
    return np.array(array_Z)

In [159]:
#Other code for Gibbs_sampler_z_posterior
def compute_z_posterior2(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T):
    """Compute \pi(z|Y, U, X, \phi, R^2, q) mass function

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples

    Returns:
        float: value of \pi(z|Y, U, X, \phi, R^2, q)
    """
    s_z=np.sum(Z)
    #print("q  = ")
    #print(q)
    #print("s_z = ")
    #print(s_z)
    #print(k)
    #print(" ")
    #print(gamma2)
    #print("det(W_tilde) = ")
    #print(np.linalg.det(W_tilde))
    Y = compute_Y(dct["X"],beta_ini,np.random.normal(loc=0 ,scale=sigma2, size=T))
    Y_tilde = compute_Y_tilde(Y,dct["U"],dct["phi"])
    W_tilde = compute_W_tilde(X_tilde,gamma2)
    estimator_beta_tilde = compute_estimator_beta_tilde(W_tilde,X_tilde,Y_tilde)


    #print("q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5*sp.special.gamma(T/2) ")
    #print(q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-T/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde))
    #print("q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-T/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)**(-T/2)*sp.special.gamma(T/2))*(10^(100) = ")
    #print(q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-1/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)**(-1/2)*sp.special.gamma(1/2))
    return (q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-T/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)**(-1/2)*sp.special.gamma(T/2))*10**100
#We multiply by 10^(100) in order to get a non small value
def compute_zi_posterior_conditional_z_excluded_i2(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, zi, index):
    """Compute \pi(z_i|Y, X, U, \phi, R^2, q z_{-i}) for a specific zi

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        zi (int): Z_i variable
        index (int): index of Z_i variable in Z array

    Returns:
        float: value of \pi(z_i|Y, X, U, \phi, R^2, q z_{-i})
    """
    Z_i=np.copy(Z)
    Z_i[index]=zi
    z_posterior=compute_z_posterior2(q=q, k=k, Z=Z_i, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    #print("z_posterior = ")
    #print(z_posterior)
    Z_0=np.copy(Z)
    Z_0[index]=0
    z_posterior_zi_equal_0=compute_z_posterior2(q=q, k=k, Z=Z_0, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    Z_1=np.copy(Z)
    Z_1[index]=1
    z_posterior_zi_equal_1=compute_z_posterior2(q=q, k=k, Z=Z_1, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    return -+z_posterior/(z_posterior_zi_equal_0+z_posterior_zi_equal_1)




def simulated_zi_posterior_conditional_z_excluded_i2(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, index, seed):
    """Sample z_i|Y, X, U, \phi, R^2, q z_{-i} by inverse CDF method

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        index (int): index of Z_i variable in Z array
        seed (int): seed

    Returns:
        int: value of z_i|Y, X, U, \phi, R^2, q z_{-i} in {0,1}
    """
    #np.random.seed(seed)
    proba_success=np.abs(compute_zi_posterior_conditional_z_excluded_i2(q=q, k=k, Z=Z, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, zi=1, index=index))
    #print("proba_success = ")
    #print(proba_success)
    u=np.random.uniform(0,1);
    #print("u = ")
    #print(u)
    if u<1-proba_success:
        return 0
    else:
        return 1



def step_gibbs_sampler_z_posterior2(q, k, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, seed, Z_t_minus_1):
    """One iteration of Gibbs sampler

    Args:
        q (float): q variable
        k (int): number of x predictors
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        seed (int): seed
        Z_t_minus_1 (_type_): Z_{t-1} variable sampled during the previous iteration (t-1 step)

    Returns:
        int:
    """
    Z_t=np.copy(Z_t_minus_1)
    for i in range(len(Z_t)):
        Z_t[i]=simulated_zi_posterior_conditional_z_excluded_i2(q=q, k=k, Z=Z_t, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, index=i, seed=seed)

    return Z_t


def gibbs_sampler_z_posterior2(q, k, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, seed, n_iter, n_variables):
    """Gibbs sampler to simulate 1*k vectors z|Y, U, X, \phi, R^2

    Args:
        q (float): q variable
        k (int): number of x predictors
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        seed (int): seed
        n_iter (int): number of iterations
        n_variables (int): number of variables desired

    Returns:
        np.array: array of n_variables z=(z_1,...,z_k)
    """
    array_Z=[]
    for n in range(n_variables):
        Z_0=z_ini
        Z_t=Z_0
        for t in range(n_iter):
            Z_t=step_gibbs_sampler_z_posterior2(q=q, k=k, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, seed=seed, Z_t_minus_1=Z_t)
        array_Z.append(Z_t)
    return np.array(array_Z)


In [104]:
Z=dct['Z']
X=dct['X']
U=dct['U']
Y=dct['Y']
q=dct["q"]
gamma2=dct["gamma2"]
beta=dct["Beta"]
phi=dct["phi"]
sigma2=dct["sigma2"]

In [160]:
#X_tilde=compute_X_tilde(X=X,beta=beta)
X_tilde=compute_X_tilde(X=X,beta=beta_ini)
W_tilde=compute_W_tilde(X_tilde=X_tilde, gamma2=gamma2)
Y_tilde=compute_Y_tilde(Y=Y, U=U, phi=phi)
estimator_beta_tilde=compute_estimator_beta_tilde(W_tilde=W_tilde, X_tilde=X_tilde, Y_tilde=Y_tilde)

sample_posterior_z=gibbs_sampler_z_posterior(q=q,
                                   k=k,
                                   gamma2=gamma2,
                                   W_tilde=W_tilde,
                                   Y_tilde=Y_tilde,
                                   estimator_beta_tilde=estimator_beta_tilde,
                                   T=T,
                                   seed=seed,
                                   n_variables=2,
                                   n_iter=10000)

print(sample_posterior_z)

<ipython-input-158-6605b15820ef>:50: RuntimeWarning:

invalid value encountered in double_scalars



KeyboardInterrupt: ignored

In [161]:
#Gibbs sampler ran with the other code
sample_posterior_z=gibbs_sampler_z_posterior2(q=q,
                                   k=k,
                                   gamma2=gamma2,
                                   W_tilde=W_tilde,
                                   Y_tilde=Y_tilde,
                                   estimator_beta_tilde=estimator_beta_tilde,
                                   T=T,
                                   seed=seed,
                                   n_variables=2,
                                   n_iter=10)

print(sample_posterior_z)

[[1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1.
  1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
  0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 0.]]


In [156]:
X_tilde=compute_X_tilde(X=X,beta=beta)
W_tilde=compute_W_tilde(X_tilde=X_tilde, gamma2=gamma2)
Y_tilde=compute_Y_tilde(Y=Y, U=U, phi=phi)
estimator_beta_tilde=compute_estimator_beta_tilde(W_tilde=W_tilde, X_tilde=X_tilde, Y_tilde=Y_tilde)

sample_posterior_z=gibbs_sampler_z_posterior2(q=q,
                                   k=k,
                                   gamma2=gamma2,
                                   W_tilde=W_tilde,
                                   Y_tilde=Y_tilde,
                                   estimator_beta_tilde=estimator_beta_tilde,
                                   T=T,
                                   seed=seed,
                                   n_variables=2,
                                   n_iter=10)

print(sample_posterior_z)

[[1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1.
  0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
  0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
  1. 0. 1. 1.]]


### 4. Draw from the conditional posterior of $\sigma^2$

In [ ]:
def sample_sigma2_posterior(Y_tilde, estimator_beta_tilde, X_tilde, gamma2, T, n_variables, seed):
    """Sample random variables sigma^2|Y, U, X, ϕ, R2, q, z

    Args:
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        X_tilde (np.array): T*s(z) \Tilde{X} matrix
        gamma2 (float): gamma^2 variable
        T (int): number of samples
        n_variables (int): number of variables desired
        seed (int): seed

    Returns:
        _type_: n_variables sigma^2|Y, U, X, ϕ, R2, q, z
    """
    np.random.seed(seed)
    I_s_z=np.identity(X_tilde.shape[1])
    inverse_gamma_dist = sp.stats.invgamma(T/2, scale=0.5*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@(X_tilde.T@X_tilde+(1/gamma2)*I_s_z)@estimator_beta_tilde))
    return inverse_gamma_dist.rvs(size=n_variables)

In [ ]:
array_sigma2_posterior=sample_sigma2_posterior(Y_tilde=Y_tilde,
                                               estimator_beta_tilde=estimator_beta_tilde,
                                               X_tilde=X_tilde,
                                               gamma2=gamma2,
                                               T=T,
                                               n_variables=2,
                                               seed=seed)
print(array_sigma2_posterior)

[63387.13951352 66316.41976906]


### 5. Draw from the conditional posterior of $\tilde{\beta}$

In [141]:
def sample_beta_tilde_posterior(X_tilde, Y, U, phi, sigma2, gamma2, n_variables, seed):
    """Sample 1*s(z) random vectors \tilde{β}|Y, U, X, ϕ, R2, q, sigma^2, z

    Args:
        X_tilde (np.array): T*s(z) \Tilde{X} matrix
        Y (np.array): T*1 vector of target
        U (np.array): T*l matrix of predictors
        phi (np.array): 1*l phi vector
        sigma2 (float): sigma^2
        gamma2 (float): gamma^2
        n_variables (int): number of variables
        seed (int): seed

    Returns:
        _type_: n_variables of 1*l random vectors
    """
    np.random.seed(seed)
    I_s_z=np.identity(X_tilde.shape[1])
    return np.random.multivariate_normal(np.linalg.inv((1/gamma2)*I_s_z+X_tilde.T@X_tilde)@X_tilde.T@(Y-U@phi),sigma2*np.linalg.inv((1/gamma2)*I_s_z+X_tilde.T@X_tilde), n_variables)

In [142]:
array_beta_tilde_posterior=sample_beta_tilde_posterior(X_tilde=X_tilde,
                                                       Y=Y,
                                                       U=U,
                                                       phi=phi,
                                                       sigma2=sigma2,
                                                       gamma2=gamma2,
                                                       n_variables=2,
                                                       seed=seed)

print(array_beta_tilde_posterior)

[[-69.56279735   1.93176162  13.18661359  11.86696936  -9.15351123]
 [-66.97343992  -2.37979316  11.64062591  10.53476085  -9.74114333]]


* $l=0$ dans l'énoncé donc, $U=0$: bizarre comme certaines variances dépendent de U (celle du posterior de $\phi$) et valent donc 0
* Doute sur la génération de $\beta$ et $Z$: en théorie $\beta$, suit un mélange de lois (gaussien + dirac). Or, dans l'énoncé on demande explicitement de mettre $k-s$ élements de $\beta$ à 0 et les autres composantes suivent une normale centrée réduite. De plus, $Z_j$ suit une loi de bernouilli de param_tre $q$. Or, dans ce cas, il n'y a pas de simulation aléatoire de $Z$. On regarde juste les composantes de $\beta$ non-nulles. Je trouve ça étrange.